In [38]:
import pandas as pd
from glob import glob

In [39]:
data = glob('*.xlsx')
data

['5_문어체_조례.xlsx',
 '2_대화체.xlsx',
 '1_구어체(2).xlsx',
 '1_구어체(1).xlsx',
 '3_문어체_뉴스(2).xlsx',
 '3_문어체_뉴스(3).xlsx',
 '3_문어체_뉴스(1).xlsx',
 '4_문어체_한국문화.xlsx',
 '3_문어체_뉴스(4).xlsx',
 '6_문어체_지자체웹사이트.xlsx']

In [ ]:
# 전체 데이터 중 문어체_조레, 문어제_지자체웝사이트 파일 제외하고 데이터 프레임 생성

df = pd.DataFrame(columns=['원문', '번역문'])

file_list = ['2_대화체.xlsx',
             '1_구어체(2).xlsx',
             '1_구어체(1).xlsx',
             '3_문어체_뉴스(2).xlsx',
             '3_문어체_뉴스(3).xlsx',
             '3_문어체_뉴스(1).xlsx',
             '4_문어체_한국문화.xlsx',
             '3_문어체_뉴스(4).xlsx']

for data in file_list:
    temp = pd.read_excel(data)
    df = pd.concat([df, temp[['원문', '번역문']]])

In [ ]:
# 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

# 토크나이저 만들기

In [ ]:
from torchtext import data
from konlpy.tag import Okt

tokenizer = Okt()

In [ ]:
def tokenize_kor(text):
    """
    한국어를 tokenizer해서 단어들을 리스트로 만든 후 reverse하여 반환
    """
    return [text_ for text_ in tokenizer.morphs(text)][::-1]


def tokenize_en(text):
    """
    영어를 split tokenizer해서 단어들을 리스트로 만드는 함수

    """
    return [text_ for text_ in text.split()]

# 필드 정의


SRC = data.Field(tokenize=tokenize_kor,
                 init_token='<sos>',
                 eos_token='<eos>', batch_first=True, lower=True)

TRG = data.Field(tokenize=tokenize_en,
                 init_token='<sos>',
                 eos_token='<eos>', batch_first=True,
                 lower=True)

# 데이터셋 만들기 (전체 데이터중 10만개만 사용)

In [51]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [52]:
from sklearn.model_selection import KFold
# 우선 전제 데이터중 10만개만 사용
df_ = df_shuffled[:100000]

train_df = df_[:95000]
test_df = df_[95000:]

In [53]:
print('trn size: ', len(train_df))
print('test size: ', len(test_df))

trn size:  95000
test size:  5000


In [54]:
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [55]:
from torchtext.data import TabularDataset

train_data, test_data = TabularDataset.splits(
    path='', train='train_df.csv', test='test_df.csv', format='csv',
    fields=[('원문', SRC), ('번역문', TRG)], skip_header=True)

In [56]:
# 학습 데이터와 검증데이터셋 분리
train_data, validation_data = train_data.split(
    split_ratio=0.8, random_state=random.seed(323))

In [57]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('검증 샘플의 개수 : {}'.format(len(validation_data)))

훈련 샘플의 개수 : 76000
검증 샘플의 개수 : 19000


# Vocab 생성

In [58]:
# 말뭉치 생성
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [59]:
print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 40291
len(TRG): 39492


# data loader

In [60]:
# data loader 생성
from torchtext.data import Iterator

# 하이퍼파라미터
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)
batch_size = 128
lr = 0.001
EPOCHS = 20

train_iterator = Iterator(dataset=train_data, batch_size=batch_size)
valid_iterator = Iterator(dataset=validation_data, batch_size=batch_size)
test_iterator = Iterator(dataset=test_data, batch_size=batch_size)

cuda:1


In [61]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_iterator)))
print('검증 데이터의 미니 배치 수 : {}'.format(len(valid_iterator)))

훈련 데이터의 미니 배치 수 : 594
검증 데이터의 미니 배치 수 : 149


# multi head attention
1. query 행렬과 key 행렬간의 내적을 계산
2. 1번 값에 key 행렬의 차원의 제곱근으로 나눔
3. 2번 값에 소프트맥스 함수를 적용해 정규화 진행
4. 3번 값에 value 행렬을 곱해 attention 출력

In [62]:
import torch.nn as nn


class MultiHeadAttentionLayer(nn.Module):
    """
    encoder와 decoder의 multi head attention 부분

    임베딩 된 sequence + positional encoding (or 이전 layer의 output) 을 이용해 
    self attention 을 수행하고 다음 layer(residual, normalization)로 보냄

    attribute 
    ---
    hidden_dim : int
        임베딩 차원
    n_heads : int
        헤드(head)의 개수: 서로 다른 어텐션(attention)의 수
    dropout_ration : float
        dropout 비율
    device : 
        모델 학습과 추론에 사용할 device(gpu or cpu)


    """

    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim  # 임베딩 차원
        self.n_heads = n_heads  # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        # 각 헤드(head)에서의 임베딩 차원 -> 각 헤드의 차원 * 헤드의 hidden dim  = input hidden dim 이여야함
        self.head_dim = hidden_dim // n_heads

        self.fc_q = nn.Linear(hidden_dim, hidden_dim)  # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim)  # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim)  # Value 값에 적용될 FC 레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask=None):
        """
        파라미터 
        ---

        qurey :  [batch_size, query_len, hidden_dim]
            input 데이터의 qurey 벡터
        key :  [batch_size, query_len, hidden_dim]
            input 데이터의 key 벡터
        value :  [batch_size, query_len, hidden_dim]
            input 데이터의 key 벡터
        mask : boolen
            모델 추론시 다음 단어 정보를 가리기위한 mask 행렬

        returns 
        ---

        x : [batch size, seq_len, hidden_size]
            input과 차원이 같이야 여러 layer를 쌓을수 있음.
        attention :
            현재 layer의 attention score
       """

        batch_size = query.shape[0]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        Q = Q.view(batch_size, -1, self.n_heads,
                   self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads,
                   self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads,
                   self.head_dim).permute(0, 2, 1, 3)

        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]

        # Attention Energy 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / \
            self.scale  # 기울기 소실 방지

        # energy: [batch_size, n_heads, query_len, key_len]

        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기
            energy = energy.masked_fill(mask == 0, -1e10)

        # 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim=-1)

        # attention: [batch_size, n_heads, query_len, key_len]

        # 여기에서 Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)

        # x: [batch_size, n_heads, query_len, head_dim]

        x = x.permute(0, 2, 1, 3).contiguous()

        # x: [batch_size, query_len, n_heads, head_dim]

        x = x.view(batch_size, -1, self.hidden_dim)

        # x: [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)

        # x: [batch_size, query_len, hidden_dim]

        return x, attention

# feedforward layer
피드포워드 레이어는 2개의 dense layer과 ReLU함수로 구성됨.

In [63]:
class PositionwiseFeedforwardLayer(nn.Module):
    """

    attribute 
    ---
    hidden_dim : int
        임베딩 차원
    pf_dim : int
        feedforwa layer시킬 차원
    dropout_ration : float
        dropout 비율

    """

    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        """
        파라미터 
        ---
        x : [batch_size, seq_len, hidden_dim]
            multi head attention을 거쳐나온 output 값

        returns 
        ---
        x : [batch_size, seq_len, hidden_dim]
            다음 layer로 보내기 값 출력
        """

        x = self.dropout(torch.relu(self.fc_1(x)))

        # x: [batch_size, seq_len, pf_dim]

        x = self.fc_2(x)

        # x: [batch_size, seq_len, hidden_dim]

        return x

# transformer 의 encoder
1. 입력 sequence를 임베딩 + positional embedding 값을 첫번째 인코더의 입력값으로 넣음
2. 인코더 1은 입력값을 받아 multi head attention을 통해 attention 행렬 값 출력
3. 어텐션 행렬을 FeedForward로 입력
4. 그 다음 인코더 1의 출력값을 그 위에 있는 인코더(인코더 2)에 입력
5. 인코더2는 위 과정을 반복.

In [64]:
class EncoderLayer(nn.Module):
    """
    Encoder 내부를 구축하는 EncoderLayer

    attribute 
    ---
    hidden_dim : int
        임베딩 차원
    pf_dim : int
        feedforward 시킬 차원
    dropout_ration: float
        dropout 비율
    device : cuda or cpu
        device 종류 
    """

    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(
            hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(
            hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, src, src_mask):
        """
        파라미터
        ---
        src: [batch_size, src_len, hidden_dim]
            input 데이터
        src_mask: [batch_size, src_len]
            필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절
        """

        # self attention
        # 하나의 입력값을 통해 query, key, value 값 생성 , decoder는 다름.
        _src, _ = self.self_attention(src, src, src, src_mask)

        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # src: [batch_size, src_len, hidden_dim]

        # position-wise feedforward
        _src = self.positionwise_feedforward(src)

        # dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))

        # src: [batch_size, src_len, hidden_dim]

        return src

In [65]:
class Encoder(nn.Module):
    """
    transformer 모델의 encoder
    encoder layer를 여러층 쌓는다

    attribute 
    ---
    input_dim : int
        input 데이터의 임베딩 차원    
    hidden_dim : int
        임베딩 차원
    n_layers: int
         encoder layer의 갯수
    n_heads : int
        multi head attention 의 head 갯수
    pf_dim : int
        feedforward 시킬 차원
    dropout_ration: float
        dropout 비율
    device : cuda or cpu
        device 종류 
    """

    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)
        # layer를 쌓음
        self.layers = nn.ModuleList([EncoderLayer(
            hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):

        # src: [batch_size, src_len]
        # src_mask: [batch_size, src_len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(
            0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, src_len]

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout(
            (self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        # src: [batch_size, src_len, hidden_dim]

        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.layers:
            src = layer(src, src_mask)

        # src: [batch_size, src_len, hidden_dim]

        return src  # 마지막 레이어의 출력을 반환

# transformer의 decoder
1. 디코더에 대한 입력 문장을 임베딩 행렬로 변환한 다음 위치 인코딩 정보를 추가하고 디코더(디코더 1)에 입력
2. 디코더는 입력을 가져와 마스크된 멀티 헤드 어텐션 레이어에 보내고, 출력으로 어텐션 행렬 M을 반환
3. 어텐션 행렬M, 인코딩 표현 R을 입력받아 멀티헤드 어텐션 레이어(인코더- 디코더 어텐션 레이어)에 값을 입력하고, 출력으로 새로운 어텐션 행렬 생성
4. 어텐션 행렬을 피드포워드 레이어에 입력하여 값을 출력
5. 디코더 1의 출력값을 다음 디코더(디코터 2)의 입력값으로 사용
6. 디코더 2는 해당 내용 반복하며, 타깃 문장에 대한 디코더 표현을 반환.

In [66]:
class DecoderLayer(nn.Module):
    """
    decoder 내부를 구축하는 decoderLayer

    attribute 
    ---
    hidden_dim : int
        임베딩 차원
    n_heads : int 
        multi head attention의 head의 갯수
    pf_dim : int
        feedforward 시킬 차원
    dropout_ration: float
        dropout 비율
    device : cuda or cpu
        device 종류 
    """

    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(
            hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(
            hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(
            hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):
        """
        파라미터
        ---
        trg: [batch_size, trg_len, hidden_dim]
            target 텍스트 (번역된 단어)

        enc_src: [batch_size, src_len, hidden_dim]
            encoder의 출력

        trg_mask: [batch_size, trg_len]

        src_mask: [batch_size, src_len]

        returns 
        ---
        trg : 
            번역된 단어 
        attention : 
            attention 행렬        
        """

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(
            trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

In [67]:
class Decoder(nn.Module):
    """
    transformer 모델의 Decoder
    Decoder layer를 여러층 쌓는다

    attribute 
    ---
    output_dim : int
        output 데이터의 임베딩 차원    
    hidden_dim : int
        임베딩 차원
    n_layers: int
         encoder layer의 갯수
    n_heads : int
        multi head attention 의 head 갯수
    pf_dim : int
        feedforward 시킬 차원
    dropout_ration: float
        dropout 비율
    device : cuda or cpu
        device 종류 
    """

    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(
            hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(
            0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, trg_len]

        trg = self.dropout(
            (self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]

        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention

In [68]:
class Transformer(nn.Module):
    """
    taransformer 모델 
    encoder와 decoder로 구성됨

    """

    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        trg_sub_mask = torch.tril(torch.ones(
            (trg_len, trg_len), device=self.device)).bool()

        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

# 모델 학습


In [69]:
# 하이퍼 파라미터 설정
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [70]:
# 모델 정의
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

In [71]:
device = torch.device('cuda:1')
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS,
              ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS,
              DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)


# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
# 가중치 초기화
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


model.apply(initialize_weights)

In [73]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [74]:
def train(model, iterator, optimizer, criterion, clip):
    """
    모델 학습 함수
    """
    model.train()  # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):

        src = batch.원문.to(device)
        trg = batch.번역문.to(device)

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:, :-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:, 1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward()  # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [75]:
def evaluate(model, iterator, criterion):
    """
    모델 평가 함수
    """
    model.eval()  # 평가 모드
    epoch_loss = 0

    # 전체 평가 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.원문.to(device)
        trg = batch.번역문.to(device)

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:, :-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:, 1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [76]:
import math
import time


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [77]:
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()  # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()  # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(
        f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 1m 15s
	Train Loss: 6.103 | Train PPL: 447.402
	Validation Loss: 5.132 | Validation PPL: 169.315
Epoch: 02 | Time: 1m 15s
	Train Loss: 4.932 | Train PPL: 138.722
	Validation Loss: 4.632 | Validation PPL: 102.688
Epoch: 03 | Time: 1m 15s
	Train Loss: 4.327 | Train PPL: 75.738
	Validation Loss: 4.294 | Validation PPL: 73.234
Epoch: 04 | Time: 1m 15s
	Train Loss: 3.801 | Train PPL: 44.756
	Validation Loss: 4.110 | Validation PPL: 60.972
Epoch: 05 | Time: 1m 15s
	Train Loss: 3.356 | Train PPL: 28.684
	Validation Loss: 4.015 | Validation PPL: 55.427
Epoch: 06 | Time: 1m 15s
	Train Loss: 2.975 | Train PPL: 19.592
	Validation Loss: 4.006 | Validation PPL: 54.931
Epoch: 07 | Time: 1m 15s
	Train Loss: 2.646 | Train PPL: 14.104
	Validation Loss: 4.042 | Validation PPL: 56.942
Epoch: 08 | Time: 1m 15s
	Train Loss: 2.367 | Train PPL: 10.662
	Validation Loss: 4.099 | Validation PPL: 60.308
Epoch: 09 | Time: 1m 15s
	Train Loss: 2.129 | Train PPL: 8.403
	Validation Loss: 4.194 | Val